In [ ]:
# Import Libraries
import time

from ImportData2D import get_label, getFilename, get_file_name
from stlSlicer import getSlicer, slicecoor, rotatestl
from imgSave import saveplot
import numpy as np

np.random.seed(125)


In [ ]:
label = getLabel("Occ_Sum", "median", True, False,False)
print(type(label[0]))
label = getLabel("Occ_Sum", "median", True, True,False)
print(type(label[0][0]))

In [ ]:
import os
test_name = "prep_test.tfrecords"
test_name = os.path.join("data", test_name)
print(test_name)

In [ ]:
import os
A = './data/preparation_181_data_test_eval_address.txt'
print(os.path.abspath(A))
print(os.path.isfile(A))

In [ ]:
example_grouped_address = ['/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_0_84140','/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_95138_0','/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_304138_45']
grouped_address = [1,2,3]
# define an empty list
train_address = []

# open file and read the content in a list
with open(A, 'r') as filehandle:  
    for line in filehandle:
        # remove linebreak which is the last character of the string
        current_name = line[:-1]
        # print(current_name)
        for i, name in enumerate(example_grouped_address):
            
            if current_name in name:
                train_address.append(grouped_address[i])
                grouped_address.remove(grouped_address[i])
                example_grouped_address.remove(example_grouped_address[i])
                

In [ ]:
print(train_address)

In [ ]:
import ast
B = ast.literal_eval(error_file_names[0])
print(B[0])

In [ ]:
train_address = ['abc_0_0','abc_0_90','abc_0_45','abc_0_135','abc_1_0','abc_1_45','abc_1_90','abc_1_70']

In [ ]:
train_address.sort()
print([s for s in train_address if 'abc_0' in s])

In [ ]:
import tensorflow as tf
import csv
A = {'learning_rate':0.00009123,'keep_prob':0.25, 'activation':tf.nn.relu, 'channels':[16,16,32]}
# A = {'a':'b','c':'d'}

with open("output"+".csv", "w") as csvfile:
    writer = csv.writer(csvfile)
    for key, val in A.items():
        writer.writerow([key, val])


In [ ]:
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
# You can add the parameter data_home to wherever to where you want to download your data
import tensorflow as tf
import numpy as np
((train_data, train_labels),(eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()
train_data = train_data / np.float32(255)
train_data = train_data.reshape(train_data.shape[0], -1)
print(train_data.shape)
train_labels = train_labels.astype(np.int32)  # not required
print(train_labels.shape)
# test_size: what proportion of original data is used for test set
train_img, test_img, train_lbl, test_lbl = train_test_split(
    train_data, train_labels, test_size=1/7.0, random_state=0)
print(train_img.shape)

In [ ]:
# Standardizing the Data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(train_img)

# Apply transform to both the training set and the test set.
train_img = scaler.transform(train_img)
test_img = scaler.transform(test_img)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
pca = PCA(.95)
pca.fit(train_img)
pca.n_components_
train_img = pca.transform(train_img)
test_img = pca.transform(test_img)

# all parameters not specified are set to their defaults
# default solver is incredibly slow thats why we change it
# solver = 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')

logisticRegr.fit(train_img, train_lbl)

# Returns a NumPy Array
# Predict for One Observation (image)
logisticRegr.predict(test_img[0].reshape(1,-1))

# Predict for Multiple Observations (images) at Once
logisticRegr.predict(test_img[0:10])

In [ ]:
score = logisticRegr.score(test_img, test_lbl)
print(score)

In [ ]:
data_path = './data/preparation_181_data_Taper_Sum_median_eval.tfrecords'
arrays = get_data_from_path(data_path)
# example, label = get_data_from_path(data_path)
# print(example[0])
print((arrays[1]))

In [ ]:
ex, label = get_data_from_path(data_path)
print(label)

In [ ]:
ex, label = get_data_from_path(data_path)
print(label)

In [ ]:
import csv

csvData = [['Person', 'Age'], ['Peter', '22'], ['Jasmine', '21'], ['Sam', '24']]
print(type(csvData))
print((csvData))
print(type(csvData[0]))
print((csvData[0]))
'''
with open('person.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csvData)

csvFile.close()'''

In [ ]:
csvData = [(-0.53103447, [1e-05, 0.0, '1', 1]), (-0.53103447, [4.315202424267864e-05, 0.0, '1', 1]), (-0.53103447, [6.388961292348475e-05, 0.04720173848853115, '1', 4]), (-0.53103447, [0.0001540877645803547, 0.35419707255064065, '1', 1])]
newData = []
for i in csvData:
    data = {'value': i[0],
           'learning_rate': i[1][0],
           'dropout_rate': i[1][1],
           'activation': i[1][1],
           'channels': i[1][1],}
    newData.append(data)
print(type(csvData))
print((csvData))
#print(type(csvData[0]))
# print((csvData[0]))


with open('person.csv', 'w') as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames = ['value','learning_rate','dropout_rate', 'activation', 'channels'])
    writer.writeheader()
    writer.writerows(newData)



In [ ]:
print("Func vals: %s \n" % search_result.func_vals)
print("Hyper para: %s \n" % search_result.x_iters)
print("Combined: %s" % list(zip(search_result.func_vals, search_result.x_iters)))

In [ ]:
searched_parameter_2 = (list(zip(search_result.func_vals, search_result.x_iters)))
i = searched_parameter_2[0]
data_2 = {field_name[0]: i[0] * -1,
                field_name[1]: i[1][0],
                field_name[2]: i[1][1],
                field_name[3]: i[1][1],
                field_name[4]: i[1][1]}
print(data_2)

In [23]:
stl_points = points
new_stl_points = []
for stl_point_sample in stl_points:  # stl_point_sample is one example of stl_points
    new_points_sample = []
    for stl_point_image in stl_point_sample:  # stl_point_image is one degree of cross-section
        difference = stl_point_image[1:, :] - stl_point_image[0:-1, :]  # Find difference between each position
        new_points_sample.append(difference)
    new_stl_points.append(new_points_sample)

In [24]:
print(type(new_stl_points))
print(len(new_stl_points))
print(type(new_stl_points[0]))
print(len(new_stl_points[0]))
print(type(new_stl_points[0][0]))
print(np.shape(new_stl_points[0][0]))

<class 'list'>
362
<class 'list'>
4
<class 'numpy.ndarray'>
(352, 2)


In [26]:
print(lbl_name)
print(err_name)

['304101', '304102', '304103', '304104', '304105', '304106', '304107', '304108', '304109', '304110', '304111', '304112', '304113', '304114', '304115', '304116', '304117', '304118', '304119', '304120', '304122', '304123', '304124', '304125', '304126', '304127', '304128', '304129', '304130', '304131', '304132', '304133', '304134', '304135', '304136', '304137', '304138', '304139', '304140', '304141', '304142', '304143', '304144', '304145', '304146', '304147', '304148', '304149', '304150', '304152', '304153', '304154', '304155', '305101', '305102', '305103', '305104', '305105', '305106', '305107', '305108', '305109', '305110', '305111', '305112', '305113', '305114', '305115', '305116', '305117', '305118', '305119', '305120', '305121', '305122', '305123', '305124', '305125', '305126', '305127', '305128', '305129', '305130', '305131', '305132', '305133', '305134', '305135', '305136', '305137', '305138', '305139', '305140', '305141', '305201', '305202', '305203', '305204', '305205', '305206',

In [48]:
import os
# File name will be [tfrecord_name]_train_Taper_sum_median
tfrecord_file_name = "preparation_181_data"
# tfrecord_name = "original_preparation_data"
# csv_name = "../global_data/Ground Truth Score_50.csv"
# Directory of image
dataset_folder_dir = "./data/cross_section"
 
file_name = './data/preparation_181_data_train_address.txt'
name = []
if os.path.isfile(file_name):  # Check if file exist
    with open(file_name) as f:
        name = f.read().splitlines()
    '''
    with open(file_name, 'r') as filehandle:
        for line in filehandle:
            name.append(line)
            
            # remove linebreak which is the last character of the string
            # current_name = line[:-1]
            
            # check if it exist in grouped exist, if found, put in train_address
            for i, name in enumerate(example_grouped_address):
                if current_name in name:
                    train_address.append(grouped_address[i])
                    grouped_address.remove(grouped_address[i])
                    example_grouped_address.remove(example_grouped_address[i])
                    break
            '''
    # print("Use %s examples from previous tfrecords as training" % len(train_address))
else:
    print("Not found")

In [49]:
print(type(name))
for n in name[1:]:
    print(n)

<class 'list'>
/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_85148
/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_304141
/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_304137
/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_0_85108
/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_0_95143
/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_305109
/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_95204
/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/d

In [46]:
print (n[1] == '/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_85148')

False


In [54]:
import numpy as np
A = np.array([[1,2],[3,4],[5,6]])
print((A.tolist()))
print(A[0])

[[1, 2], [3, 4], [5, 6]]
[1 2]


In [63]:
def readjust_median_label(label, avg_data):
    possible_value = [1,3,5]
    for i, label_value in enumerate(label):
        if not (label_value in possible_value):
            # Check if value is over/under boundary, if so, choose the min/max value
            if label_value < possible_value[0]:
                label[i] = possible_value[0]
            elif label_value > possible_value[-1]:
                label[i] = possible_value[-1]
            else:
                if label_value > avg_data[i]:  # If median is more than average, round up
                    label[i] = min(filter(lambda x: x > label_value, possible_value))
                else:  # If median is less or equal to average, around down
                    label[i] = max(filter(lambda x: x < label_value, possible_value))
    return label

A = [1,3,5,2,2,4,4, 10, -2, 2]
B = [1,4,2,3,1,4,7, 15]

C = readjust_median_label(A,B)
print(C)

IndexError: list index out of range

In [4]:
import numpy as np
data = np.load('/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/coordinates/PreparationScan_1_305139_0.npy')
print(type(data))
print(data)

<class 'numpy.ndarray'>
[[-6.1000400e-03  1.8794090e-02]
 [-3.3074600e-03  1.1729170e-02]
 [-6.0030200e-03  2.1288570e-02]
 [-2.2999770e-02  4.4410420e-02]
 [-2.5365010e-02  5.7631900e-02]
 [-2.8510850e-02  5.5286940e-02]
 [-1.7658300e-02  4.3184440e-02]
 [-1.6639060e-02  4.0284720e-02]
 [-9.1302000e-03  4.4213360e-02]
 [-5.8973500e-03  3.1948890e-02]
 [-1.4419200e-03  2.8960500e-02]
 [-1.0706600e-03  4.2316640e-02]
 [ 9.6940000e-05  7.4539130e-02]
 [-3.2870000e-05  9.7903800e-03]
 [-6.0413000e-04  9.9386710e-02]
 [-4.3111000e-04  1.0148770e-02]
 [-1.4121500e-03  1.2943726e-01]
 [-4.2546000e-04  3.2664790e-02]
 [-2.0401000e-04  3.4746380e-02]
 [-8.7882000e-04  1.5819556e-01]
 [-4.7470000e-04  9.7593500e-03]
 [-4.1446000e-04  1.4830470e-02]
 [-4.2025400e-03  9.2061130e-02]
 [-7.0177400e-03  1.6337027e-01]
 [-4.8800000e-06  1.3935000e-04]
 [-1.4000000e-07  1.5998000e-04]
 [-7.5200000e-06  6.6158000e-04]
 [-4.5531000e-04  2.2673194e-01]
 [-9.6398000e-04  6.2805430e-02]
 [-1.9014700e-03  4

In [2]:
import numpy as np
A = np.array([1,2,3])
B = np.array([4,5,6])
for a,b in zip(A,B):
    print(a)
    print(b)

1
4
2
5
3
6


In [14]:
import csv
with open("./test.csv", "w") as csvfile:
    fieldnames = ['Predicted Class', 'Label', 'Confident level']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

In [13]:
import csv
var1 = [1,2]
var2 = [11,12]
var3 = [21,22]
with open("./test.csv", "a") as csvfile:
    writer = csv.writer(csvfile)
    for i,j,k in zip(var1,var2,var3):
        writer.writerow([i,j,k])